In [ ]:
import numpy as np
import pandas as pd
import datetime

In [ ]:
# Target Group: adult patients' first icu visit
df = pd.read_csv('icu_first_18.csv')
df = df.drop(df.columns[0], axis=1)

# Create ICU Id list
icu = []
for icuid in df['icustay_id']:
    icu.append(icuid)

    
# Max chloride
df = pd.read_csv('chloride.csv')
df = df[df['icustay_id'].isin(icu)]
df = df.filter(['subject_id','hadm_id','icustay_id','icu_day','chloride_max','chloride_input_meq'])
df= df.query('icu_day == 1')
# df = df.dropna(subset = ['chloride_max'])

df_chl_max = df.filter(['subject_id','hadm_id','icustay_id','chloride_max'])
df_chl_max = df_chl_max.dropna(subset = ['chloride_max'])
df_chl_input = df.filter(['subject_id','hadm_id','icustay_id','chloride_input_meq'])


df_chl_max = df_chl_max.set_index(['subject_id','hadm_id','icustay_id'])
df_chl_input = df_chl_input.set_index(['subject_id','hadm_id','icustay_id'])

# Max chloride
#df = pd.read_csv('chloride.csv')
#df = df[df['icustay_id'].isin(icu)]
#df = df.set_index(['subject_id','hadm_id','icustay_id'])
#df = df.filter(['icustay_id','icu_day','chloride_max','chloride_input_meq'])
# df = df.dropna(subset = ['chloride_max'])

# Chloride >= 110 in the second day
#df_chl = df.query('icu_day == 2')
#df_chl['chl_110'] = (df_chl['chloride_max'] >= 110).astype(int)

# Demographic
df = pd.read_csv('adm_demographics.csv')
df = df.set_index(['subject_id', 'hadm_id'])
df_demo = df.filter(['insurance','ethnicity','age','gender'])

# Weight
df = pd.read_csv('weight.csv')
df = df[df['icustay_id'].isin(icu)]
df_wt = df.filter(['icustay_id','day','weight'])

# GCS(Glasgow coma scale)
df = pd.read_csv('gcs_pan.csv')
df = df[df['icustay_id'].isin(icu)]
df_gcs = df.filter(['icustay_id','day','mingcs'])

# First-day Vitals(Merge with GCS and Weight)
df = pd.read_csv('vitals_pan.csv')
df = df[df['icustay_id'].isin(icu)]
df = df.set_index(['subject_id','hadm_id','icustay_id'])
df_vitals = df.filter(['day', 'heartrate_max','sysbp_min','diasbp_min','resprate_max','spo2_min','tempc_max'])
df_vitals = df_vitals.reset_index()
df_vitals = df_vitals.merge(df_wt, on = ['icustay_id','day']).merge(df_gcs, on = ['icustay_id','day'])
df_vitals = df_vitals.query('(day == 1) & (0 < heartrate_max <= 250) &(40<sysbp_min <190) &(20<diasbp_min<150) &(6<resprate_max<80) &(0<spo2_min<100)&(30<tempc_max<44)&(30<weight<500)&(3<mingcs<15)'
                           )
df_vitals = df_vitals.set_index(['subject_id','hadm_id','icustay_id'])

# Medication from prescription table
df = pd.read_csv('medication.csv')
df_med = df.drop(df.columns[[0,3]], axis=1)
df = pd.read_csv('icu_first_18.csv')
df = df.drop(df.columns[0], axis=1)
df = df_med.merge(df, on = ['subject_id','hadm_id'])
df['taken'] = ((pd.to_datetime(df['intime']) <= pd.to_datetime(df['startdate'])) & 
               ((pd.to_datetime(df['startdate']) <= (pd.to_datetime(df['intime']) + datetime.timedelta(days=1)))))
df = df.filter(['subject_id','hadm_id','icustay_id','drug','taken'])
df_med = df.set_index(['subject_id','hadm_id','icustay_id'])
df_med = df_med.groupby(['subject_id','hadm_id','icustay_id','drug']).any()
df_med = df_med['taken'].unstack()
df_med = (df_med * 1).fillna(0)

# Intake and output
df = pd.read_csv('fluid_io.csv')
df = df[df['icustay_id'].isin(icu)]
df = df.query('icu_day == 1')
df_fluid = df.set_index(['subject_id','hadm_id','icustay_id'])
df_fluid.head()

# Diagnosis Code(group by elixhauser)
df = pd.read_csv('comorbidities_history_all.csv')
df_icd = df.drop(df.columns[[0, -1, -2]], axis=1)
df_icd = df_icd.set_index(['hadm_id'])

# Labs
df = pd.read_csv('labs1.csv')
df = df.drop(df.columns[0], axis=1)
df = df[df['icustay_id'].isin(icu)]
df_labs = df.query('day == 1 & 0.1<lactate_max<40 & 90<sodium_max<190 & 1.5<potassium_max<15 & 1<albumin_max<6.5 & 5<bicarbonate_min<65 & 0.1<creatinine_max<47 & 66<chloride_max<170 & 18<glucose_max<2500 & 2<hemoglobin_min<21 & 5<platelet_min<1600  & 0.5<inr_max<20 & 1<bun_max<280 & 0.1<wbc_max<680 & 0.1<bilirubin_min<80 & 1.5<calcium_total_min<18 & 0.2<calcium_ionized_min<2.5 & 2<ast_sgot_max<25000 & 3<amylase_max<25000 &  1<lipase_max<25000 & 0.1<c_reactive_protein_max<300')
df_labs = df_labs.set_index(['subject_id','hadm_id','icustay_id'])

# Interventions
df = pd.read_csv('interventions.csv')
df = df.query('day == 1')
df_int = df.drop(df.columns[[0]], axis=1)
df_int = df_int.set_index(['subject_id', 'hadm_id', 'icustay_id'])
df_int = df_int.filter(['label'])
df_int['taken'] = 1
df_int = df_int.groupby(['subject_id','hadm_id','icustay_id','label']).any()
df_int = df_int['taken'].unstack()
df_int = (df_int * 1).fillna(0)

In [ ]:
demo = ['age', 'gender', 'ethnicity']
vitals = ['heartrate_max', 'sysbp_min', 'diasbp_min', 'resprate_max', 'weight', 'mingcs']
medication = ['norepinephrine']
fluid = ['fluid_net_input_ml']
diag = ['congestive_heart_failure',
       'cardiac_arrhythmias', 'valvular_disease', 'pulmonary_circulation',
       'peripheral_vascular', 'hypertension', 'paralysis','other_neurological',
        'chronic_pulmonary', 
       'diabetes_complicated',  'renal_failure','solid_tumor', 'obesity',
        'fluid_electrolyte', 'drug_abuse','depression'
       ]
labs = [ #'chloride_max',
    'sodium_max', 'bicarbonate_min']
interventions = ['EPAP','IPAP','LPM','MeanAirwayPressure']

In [ ]:
df_demo = df_demo[demo]
df_vitals = df_vitals[vitals]
df_med = df_med[medication]
df_fluid = df_fluid[fluid]
df_icd = df_icd[diag]
df_labs = df_labs[labs]
df_int = df_int[interventions]

In [ ]:
df = df_chl_max.join(df_demo, how = 'inner') \
.join(df_chl_input, how = 'left') \
.join(df_fluid, how = 'left') \
.join(df_icd, how = 'left') \
.join(df_labs, how = 'left') \
.join(df_int, how='left') \
.join(df_vitals, how='left') \
.join(df_med, how = 'left') 

In [ ]:
df['gender'] = (df['gender'] == 'F').astype(int)
df = pd.concat([df, pd.get_dummies(df['ethnicity'])], axis=1)
df = df.drop(['ethnicity', 'OTHER'], axis=1)
#filter the hyperchloremic patients
df = df.query('chloride_max < 110')

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, train_size=0.7, random_state=60611)
train.to_csv('train_data.csv')
test.to_csv('test_data.csv')

In [ ]:
# fill in the missing value in training set with the training median
import psycopg2
import pandas as pd

con = psycopg2.connect(dbname = dbname, user = sqluser, password = password, host = host, port = port)

In [ ]:
train_icu = []
train = pd.read_csv('train_data.csv')
for icu_id in train['icustay_id']:
    train_icu.append(icu_id)

In [ ]:
#Labs
query = """
select * from

(SELECT ie.subject_id, ie.hadm_id, ie.icustay_id, ceiling((extract( epoch from ie.outtime - le.charttime))/60/60/24) as day
  -- the day to outtime
  , CASE

      WHEN ITEMID = 50882 THEN 'BICARBONATE'

      WHEN ITEMID = 50902 THEN  'CHLORIDE'
      WHEN ITEMID = 50806 THEN 'CHLORIDE'

      WHEN ITEMID = 50983  THEN  'SODIUM'
      WHEN ITEMID = 50824  THEN  'SODIUM'

      ELSE null
    END AS label , valuenum
  

FROM icu_18 ie

  LEFT JOIN mimiciii.labevents le
    ON le.subject_id = ie.subject_id AND le.hadm_id = ie.hadm_id
	
    AND le.ITEMID in

    (
      -- comment is: LABEL | CATEGORY | FLUID | NUMBER OF ROWS IN LABEVENTS
     

      50882, -- BICARBONATE | CHEMISTRY | BLOOD | 780733

      50902, -- CHLORIDE | CHEMISTRY | BLOOD | 795568
      50806, -- CHLORIDE, WHOLE BLOOD | BLOOD GAS | BLOOD | 48187

      50983, -- SODIUM | CHEMISTRY | BLOOD | 808489
      50824 -- SODIUM, WHOLE BLOOD | BLOOD GAS | BLOOD | 71503

    )
    AND valuenum IS NOT null AND valuenum > 0 -- lab values cannot be 0 and cannot be negative
	where    le.charttime BETWEEN (ie.intime) AND (ie.outtime)) pvt

where day <=1 
and icustay_id in 
(select icustay_id 
from mimiciii.icustays icu join
mimiciii.patients pt on icu.subject_id=pt.subject_id 
where pt.dod > icu.outtime)

"""
df1 = pd.read_sql_query(query, con)
df1 = df1[df1['icustay_id'].isin(train_icu)]

In [ ]:
df1 = df1.filter(['label','valuenum'])
df1.columns = ['vitalid','valuenum']

In [ ]:
def iqr(x):
    return x.quantile(0.75) - x.quantile(0.25)

In [ ]:
f = ['mean','median', 'std', iqr]
df2 = df1.groupby('vitalid')['valuenum'].agg(f)
df2

In [ ]:
# Vitals
query = """
select * from 

(select ie.subject_id, ie.hadm_id, ie.icustay_id, ce.charttime,
	ceiling((extract( epoch from ie.outtime- ce.charttime ))/60/60/24) as day 
  , case
    when itemid in (211,220045) and valuenum > 0 and valuenum < 300 then 'HeartRate'
    when itemid in (51,442,455,6701,220179,220050) and valuenum > 0 and valuenum < 400 then 'SysBP'
    when itemid in (8368,8440,8441,8555,220180,220051) and valuenum > 0 and valuenum < 300 then  'DiasBP'

    when itemid in (615,618,220210,224690) and valuenum > 0 and valuenum < 70 then 'RespRate'

    else null end as VitalID
      -- convert F to C
  , case when itemid in (223761,678) then (valuenum-32)/1.8 else valuenum end as valuenum

  from icu_18 ie
  left join mimiciii.chartevents ce
  on ie.subject_id = ce.subject_id and ie.hadm_id = ce.hadm_id and ie.icustay_id = ce.icustay_id
  --and ce.charttime between ie.intime and ie.intime + interval '1' day
  -- exclude rows marked as error
  and ce.error IS DISTINCT FROM 1
  where ce.itemid in
  (
  -- HEART RATE
  211, --"Heart Rate"
  220045, --"Heart Rate"

  -- Systolic/diastolic

  51, --	Arterial BP [Systolic]
  442, --	Manual BP [Systolic]
  455, --	NBP [Systolic]
  6701, --	Arterial BP #2 [Systolic]
  220179, --	Non Invasive Blood Pressure systolic
  220050, --	Arterial Blood Pressure systolic

  8368, --	Arterial BP [Diastolic]
  8440, --	Manual BP [Diastolic]
  8441, --	NBP [Diastolic]
  8555, --	Arterial BP #2 [Diastolic]
  220180, --	Non Invasive Blood Pressure diastolic
  220051, --	Arterial Blood Pressure diastolic


  -- RESPIRATORY RATE
  618,--	Respiratory Rate
  615,--	Resp Rate (Total)
  220210,--	Respiratory Rate
  224690, --	Respiratory Rate (Total)

  -- SPO2, peripheral
  646, 220277

 

  ) and ce.charttime > ie.intime)
  pvt
  
  where day <= 1 and icustay_id in
  (
  select icustay_id 
from mimiciii.icustays icu join
mimiciii.patients pt on icu.subject_id=pt.subject_id 
where pt.dod > icu.outtime
  )

"""
df = pd.read_sql_query(query, con)
df = df[df['icustay_id'].isin(train_icu)]
df = df.filter(['vitalid','valuenum'])

In [ ]:
df3 = df.groupby('vitalid')['valuenum'].agg(f)
df_final = pd.concat([df2,df3])

In [ ]:
df_final.to_csv('train_standard1.csv')

In [ ]:
# gcs
query = '''
with base as
(
  SELECT pvt.ICUSTAY_ID
  , pvt.charttime, pvt.day

  -- Easier names - note we coalesced Metavision and CareVue IDs below
  , max(case when pvt.itemid = 454 then pvt.valuenum else null end) as GCSMotor
  , max(case when pvt.itemid = 723 then pvt.valuenum else null end) as GCSVerbal
  , max(case when pvt.itemid = 184 then pvt.valuenum else null end) as GCSEyes

  -- If verbal was set to 0 in the below select, then this is an intubated patient
  , case
      when max(case when pvt.itemid = 723 then pvt.valuenum else null end) = 0
    then 1
    else 0
    end as EndoTrachFlag

  , ROW_NUMBER ()
          OVER (PARTITION BY pvt.ICUSTAY_ID ORDER BY pvt.charttime ASC) as rn

  FROM  (
  select l.ICUSTAY_ID, ceiling((extract( epoch from b.outtime- l.charttime ))/60/60/24) as day
  -- merge the ITEMIDs so that the pivot applies to both metavision/carevue data
  , case
      when l.ITEMID in (723,223900) then 723
      when l.ITEMID in (454,223901) then 454
      when l.ITEMID in (184,220739) then 184
      else l.ITEMID end
    as ITEMID

  -- convert the data into a number, reserving a value of 0 for ET/Trach
  , case
      -- endotrach/vent is assigned a value of 0, later parsed specially
      when l.ITEMID = 723 and l.VALUE = '1.0 ET/Trach' then 0 -- carevue
      when l.ITEMID = 223900 and l.VALUE = 'No Response-ETT' then 0 -- metavision

      else VALUENUM
      end
    as VALUENUM
  , l.CHARTTIME
  from mimiciii.CHARTEVENTS l

  -- get intime for charttime subselection
  inner join icu_18 b
    on l.icustay_id = b.icustay_id

  -- Isolate the desired GCS variables
  where l.ITEMID in
  (
    -- 198 -- GCS
    -- GCS components, CareVue
    184, 454, 723
    -- GCS components, Metavision
    , 223900, 223901, 220739
  )
  -- Only get data for the first 24 hours
  --and l.charttime between b.intime and b.intime + interval '1' day
  -- exclude rows marked as error
  and l.error IS DISTINCT FROM 1 and l.charttime >= b.intime
  ) pvt
  group by pvt.ICUSTAY_ID, pvt.charttime, day
)
, gcs as (
  select b.*
  , b2.GCSVerbal as GCSVerbalPrev
  , b2.GCSMotor as GCSMotorPrev
  , b2.GCSEyes as GCSEyesPrev
  -- Calculate GCS, factoring in special case when they are intubated and prev vals
  -- note that the coalesce are used to implement the following if:
  --  if current value exists, use it
  --  if previous value exists, use it
  --  otherwise, default to normal
  , case
      -- replace GCS during sedation with 15
      when b.GCSVerbal = 0
        then 15
      when b.GCSVerbal is null and b2.GCSVerbal = 0
        then 15
      -- if previously they were intub, but they aren't now, do not use previous GCS values
      when b2.GCSVerbal = 0
        then
            coalesce(b.GCSMotor,6)
          + coalesce(b.GCSVerbal,5)
          + coalesce(b.GCSEyes,4)
      -- otherwise, add up score normally, imputing previous value if none available at current time
      else
            coalesce(b.GCSMotor,coalesce(b2.GCSMotor,6))
          + coalesce(b.GCSVerbal,coalesce(b2.GCSVerbal,5))
          + coalesce(b.GCSEyes,coalesce(b2.GCSEyes,4))
      end as GCS

  from 
base	b
  -- join to itself within 6 hours to get previous value
  left join base b2
    on b.ICUSTAY_ID = b2.ICUSTAY_ID and b.rn = b2.rn+1 and b2.charttime > b.charttime - interval '6' hour and b2.charttime < b.charttime 
)
, gcs_final as (
  select gcs.*
  -- This sorts the data by GCS, so rn=1 is the the lowest GCS values to keep
  , ROW_NUMBER ()
          OVER (PARTITION BY gcs.ICUSTAY_ID
                ORDER BY gcs.GCS
               ) as IsMinGCS
  from gcs
)
select ie.SUBJECT_ID, ie.HADM_ID, ie.ICUSTAY_ID
-- The minimum GCS is determined by the above row partition, we only join if IsMinGCS=1
, day
, GCS as MinGCS
, coalesce(GCSMotor,GCSMotorPrev) as GCSMotor
, coalesce(GCSVerbal,GCSVerbalPrev) as GCSVerbal
, coalesce(GCSEyes,GCSEyesPrev) as GCSEyes
, EndoTrachFlag as EndoTrachFlag

-- subselect down to the cohort of eligible patients
from mimiciii.icustays ie
inner join gcs_final gs
  on ie.ICUSTAY_ID = gs.ICUSTAY_ID and gs.IsMinGCS = 1
where day<=1 
and ie.ICUSTAY_ID
in
(
select icustay_id 
from mimiciii.icustays icu join
mimiciii.patients pt on icu.subject_id=pt.subject_id 
where pt.dod > icu.outtime
)
ORDER BY ie.ICUSTAY_ID, day
;
'''

df2 = pd.read_sql_query(query, con)
df2 = df2[df2['icustay_id'].isin(train_icu)]

In [ ]:
stan =pd.read_csv('train_standard.csv')
stan = stan.set_index(['vitalid'])

In [ ]:
train = train.fillna(0)
test = test.fillna(0)

In [ ]:
com = pd.read_csv('comorbidities_history.csv')
com = com.drop(com.columns[0], axis=1)
com = com.set_index(['hadm_id','subject_id'])
train = train.drop(diag, 1)

In [ ]:
#impute with median
train =train.replace({'heartrate_max': {0: stan.loc['HeartRate']['median']},
               'sysbp_min': {0: stan.loc['SysBP']['median']},
               'diasbp_min': {0: stan.loc['DiasBP']['median']},
               'resprate_max': {0: stan.loc['RespRate']['median']},
               'weight': {0: stan.loc['weight']['median']},
               'mingcs': {0: df2['mingcs'].median()},
               'age': {0: stan.loc['age']['median']},
               'sodium_max': {0: stan.loc['SODIUM']['median']},
               'bicarbonate_min': {0: stan.loc['BICARBONATE']['median']},
              # 'fluid_net_input_ml': {0: stan.loc['fluid_net_input_ml']['median']},
              }) 

test =test.replace({'heartrate_max': {0: stan.loc['HeartRate']['median']},
               'sysbp_min': {0: stan.loc['SysBP']['median']},
               'diasbp_min': {0: stan.loc['DiasBP']['median']},
               'resprate_max': {0: stan.loc['RespRate']['median']},
               'weight': {0: stan.loc['weight']['median']},
               'mingcs': {0: df2['mingcs'].median()},
               'age': {0: stan.loc['age']['median']},
               'sodium_max': {0: stan.loc['SODIUM']['median']},
               'bicarbonate_min': {0: stan.loc['BICARBONATE']['median']},
             #  'fluid_net_input_ml': {0: stan.loc['fluid_net_input_ml']['median']},
              }) 

In [ ]:
train.to_csv('train.csv')
test.to_csv('test.csv')